## Setup

First let's install our required packages and set our API keys

In [3]:
from langchain_openai import AzureChatOpenAI
import os


# os.environ["AZURE_OPENAI_API_KEY"] = "2b219db0d2984f9dae28b651ab8ab3d9"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://smsh.openai.azure.com/"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
# os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"

os.environ["AZURE_OPENAI_API_KEY"] = "0c75de50975e4f278b882fe90da47f2f"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://ces.openai.azure.com"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-07-01-preview"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"

# os.environ["AZURE_OPENAI_API_KEY"] = "aa183bb914bb4858b15bed161fb47ba5"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://bxcl-prod.openai.azure.com/"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-4o"
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=0.3,
    n = 3,
    max_retries=5, request_timeout=600
)


In [4]:
model.invoke('你好')

AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}

In [4]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"  

In [5]:
from langsmith import Client

client = Client()

## Retriever
 
Let's index 3 blog posts.

In [1]:
from src.sparql_utils import *
import json 
with open("./data/WebQSP.json", 'r') as f:
    all_data = json.load(f)
i = 2
all_data[1]['Parses'][i]['TopicEntityMid']
sparql_relations = []

def abandon_rels(relation):
    """删除无效relation"""
    if relation == "type.object.type" or relation == "type.object.name" or relation.startswith("common.") or relation.startswith("freebase.") or "sameAs" in relation:
        return True
    
for entity in all_data[i]['topic_entity']:
    head_relations = get_in_relations_with_odbc(entity)

    tail_relations = get_out_relations_with_odbc(entity)
    head_relations = list(set(head_relations))
    tail_relations = list(set(tail_relations))  
    sparql_relations = head_relations + tail_relations



Freebase Virtuoso ODBC connected


In [12]:
FREEBASE_SPARQL_WRAPPER_URL = "http://localhost:3001/sparql"
FREEBASE_ODBC_PORT = 13001
sparql = SPARQLWrapper(FREEBASE_SPARQL_WRAPPER_URL)
sparql_id = """PREFIX ns: <http://rdf.freebase.com/ns/>\nSELECT DISTINCT ?tailEntity\nWHERE {\n  {\n    ?entity ns:type.object.name ?tailEntity .\n    FILTER(?entity = ns:%s)\n  }\n  UNION\n  {\n    ?entity <http://www.w3.org/2002/07/owl#sameAs> ?tailEntity .\n    FILTER(?entity = ns:%s)\n  }\n}"""
sparql_id = """PREFIX ns: <http://rdf.freebase.com/ns/>
SELECT DISTINCT ?tailEntity
WHERE {
  {
    ?entity ns:type.object.name ?tailEntity .
    FILTER(?entity = ns:%s)
  }
  UNION
  {
    ?entity <http://www.w3.org/2002/07/owl#sameAs> ?tailEntity .
    FILTER(?entity = ns:%s)
  }
  UNION
  {
    ?entity ns:common.topic.alias ?tailEntity .
    FILTER(?entity = ns:%s)
  }
}"""
entity_id = 'm.0100mt'
sparql_query = sparql_id % (entity_id, entity_id, entity_id)
execute_query_with_odbc(sparql_query)

{'El Pasas',
 'El Paso',
 'El Paso County / El Paso city',
 'El Paso, Teksas',
 'El Paso, Texas',
 'Elpaso',
 'Estrella del sud-oest',
 'Franklin',
 'La ciutat del Sol',
 'Terra de Sol',
 'Ελ Πάσο',
 'Ел Пасо',
 'Ель-Пасо',
 'Эль-Пасо',
 'אל פאסו',
 'إل باسو',
 'ال پاسو',
 'एल पासो',
 'เอลแพโซ',
 'エル・パソ',
 '艾爾帕索',
 '엘파소',
 '엘패소'}

In [23]:
get_another_entity('m.0528y98', 'type.type.instance',False)

['film.performance']

In [22]:
get_1hop_relations_with_odbc('m.0528y98')

{'film.actor.film',
 'film.film.starring',
 'film.film_character.portrayed_in_films',
 'film.performance.actor',
 'film.performance.character',
 'film.performance.character_note',
 'film.performance.film',
 'type.object.type',
 'type.type.instance'}

In [9]:
print(sparql_id)

PREFIX ns: <http://rdf.freebase.com/ns/>
SELECT DISTINCT ?tailEntity
WHERE {
  {
    ?entity ns:type.object.name ?tailEntity .
    FILTER(?entity = ns:%s)
  }
  UNION
  {
    ?entity <http://www.w3.org/2002/07/owl#sameAs> ?tailEntity .
    FILTER(?entity = ns:%s)
  }
}


In [5]:
results


In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureOpenAIEmbeddings

os.environ["AZURE_OPENAI_API_KEY"] = "2b219db0d2984f9dae28b651ab8ab3d9"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://smsh.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-03-01-preview"
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
)

from langchain.storage import LocalFileStore, RedisStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores import FAISS
store = RedisStore(redis_url="redis://localhost:6379")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
embeddings, store, namespace="openai"
)
row_string = []
with open('../data/relations', 'r') as f:
    data = f.readlines()
db = FAISS.from_texts(data, cached_embedder)
retriever = db.as_retriever()
# Add to vectorDB
# vectorstore = Chroma.from_documents(
#     documents=doc_splits,
#     collection_name="rag-chroma",
#     embedding=embeddings,
# )
# retriever = vectorstore.as_retriever()

## LLMs

In [2]:
len(sparql_relations)

52

In [14]:
### Retrieval Grader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
# LLM with function call
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
from pydantic import BaseModel, Field
from typing import List, Optional
# Please retrieve 5 relations (separated by semicolon) that contribute to the question and rate their contribution on a scale from 0 to 1 (the sum of the scores of %s relations is 1).
# Pydantic

class Rating(BaseModel):
    """Rate Model"""

    relationship: str = Field(description="The relationship name")
    rating: int = Field(description="The rate of the relevance contribution, from 0 to 5")
    reason: Optional[str] = Field(
        default=None, description="The reason for choosing the relationship and rate"
    )
class RelationOutput(BaseModel):
    """Relations list"""

    relationship: List[str] = Field(description="the most relevant 5 relationships that contribute to the question")
    rating: List[Rating] = Field(
        default=None, description="relevant 5 relations contribute to the question with their contribution rate"
    )

# Prompt
system = """You are a grader assessing relevance of retrieved graph relationships to a user question. \n 
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the relationships contain keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Please retrieve 5 relations that contribute to the question and rate their contribution from 0 to 5."""
examples = """
Relations: language.human_language.main_country; language.human_language.language_family; language.human_language.iso_639_3_code; base.rosetta.languoid.parent; language.human_language.writing_system; base.rosetta.languoid.languoid_class; language.human_language.countries_spoken_in; kg.object_profile.prominent_type; base.rosetta.languoid.document; base.ontologies.ontology_instance.equivalent_instances; base.rosetta.languoid.local_name; language.human_language.region

User question: Name the president of the country whose main spoken language was Brahui in 1980?
A: {{"relationship": [language.human_language.main_country, language.human_language.countries_spoken_in, base.rosetta.languoid.parent], "rating": [
{{"relationship": "language.human_language.main_country", "rating": 4, "reason": "This relation is highly relevant as it directly relates to the country whose president is being asked for, and the main country where Brahui language is spoken in 1980."}},
{{"relationship": "language.human_language.countries_spoken_in", "rating": 3, "reason": "This relation is also relevant as it provides information on the countries where Brahui language is spoken, which could help narrow down the search for the president."}},
{{"relationship": "base.rosetta.languoid.parent", "rating": 2, "reason": "This relation is less relevant but still provides some context on the language family to which Brahui belongs, which could be useful in understanding the linguistic and cultural background of the country in question."}}  
"""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system + examples),
        ("human", "Relations: \n\n {document} \n\n User question: {question}"),
    ]
)
structured_grader = model.with_structured_output(RelationOutput)

retrieval_grader = grade_prompt | structured_grader
question = all_data[i]['RawQuestion']

prune_relations = retrieval_grader.invoke({"question": question, "document": ';'.join(sparql_relations)})
print(prune_relations)

relationship=['location.location.contains', 'location.hud_county_place.place', 'location.hud_county_place.county', 'location.hud_county_place.countyplace_id', 'location.location.containedby'] rating=None


In [18]:
all_data[i]['topic_entity']

{'m.0xdhj': 'St. Louis Park'}

In [23]:
def get_head_entity(entity: str, relation: str, return_label=True):
        query_head = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                PREFIX : <http://rdf.freebase.com/ns/> 
                SELECT (?x0 AS ?value) WHERE {
                SELECT DISTINCT ?x0  WHERE {
                """ + '?x0' + ' :' + relation + ' :' + entity + '. ' + """
        }
        }"""
        if return_label==True:
            return [get_label_with_odbc(e) for e in execute_query(query_head)]

        return execute_query(query_head)
def get_tail_entity(entity: str, relation: str, return_label=True):
        query_tail = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                PREFIX : <http://rdf.freebase.com/ns/> 
                SELECT (?x0 AS ?value) WHERE {
                SELECT DISTINCT ?x0  WHERE {
                """ + ':' + entity + ' :' + relation +  '?x0' + '. ' + """
        }  }"""
        if return_label==True:
            return [get_label_with_odbc(e) for e in execute_query(query_tail)]
        return execute_query(query_tail)

In [16]:
relation_w_score = []

for r in prune_relations.relationship:
    if r in head_relations:
        entities = get_head_entity('m.0xdhj', r)
    else: 
        entities = get_tail_entity('m.0xdhj', r)
    relation_w_score.append({"entity":  'm.0xdhj' ,"relationship": r, "head": r in head_relations})

In [25]:
all_data[i]['topic_entity']

{'m.0xdhj': 'St. Louis Park'}

In [31]:
relation_w_score

[{'relationship': 'location.location.contains', 'head': True},
 {'relationship': 'location.hud_county_place.place', 'head': True},
 {'relationship': 'location.hud_county_place.county', 'head': False},
 {'relationship': 'location.hud_county_place.countyplace_id', 'head': False},
 {'relationship': 'location.location.containedby', 'head': True}]

In [42]:
entities=get_tail_entity('m.0xdhj', "location.hud_county_place.countyplace_id")
triplets = [f"(St. Louis Park, location.location.containedby, {entity}" for entity in entities]

In [43]:
entities

[None]

In [39]:
class TriRating(BaseModel):
    """Rate Model"""

    triplet: str = Field(description="The triplet (entity, relation, entity)")
    rating: int = Field(description="The rate of the relevance contribution, from 0 to 5")
    reason: Optional[str] = Field(
        default=None, description="The reason for choosing the triplet"
    )

class TriOutput(BaseModel):
    """Relations list"""

    relationship: List[TriRating]= Field(description="The Trirating for each triplet provided")

def entity_score():
    system = """You are a grader assessing relevance of retrieved graph triplets to a user question. \n Please score the contribution from 0 (lowest) - 5 (highest) for each triplet provided."""


    grade_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system ),
            ("human", "Triplets: \n\n {document} \n\n User question: {question}"),
        ]
    )
    structured_grader = model.with_structured_output(TriOutput)

    retrieval_grader = grade_prompt | structured_grader
    question = all_data[i]['RawQuestion']

    prune_triples = retrieval_grader.invoke({"question": question, "document": '\n'.join(triplets)})
    print(prune_triples)
entity_score()

relationship=[TriRating(triplet='(St. Louis Park, location.location.containedby, University of Phoenix-Minneapolis/St Paul Campus', rating=2, reason='Contains location information but not directly related to the county of St. Louis Park'), TriRating(triplet='(St. Louis Park, location.location.containedby, Chicago, Milwaukee, St. Paul and Pacific Depot', rating=1, reason='Not relevant to the county of St. Louis Park'), TriRating(triplet="(St. Louis Park, location.location.containedby, Toby Keith's I Love This Bar & Grill", rating=1, reason='Not relevant to the county of St. Louis Park'), TriRating(triplet='(St. Louis Park, location.location.containedby, Peavey–Haglin Experimental Concrete Grain Elevator', rating=1, reason='Not relevant to the county of St. Louis Park'), TriRating(triplet='(St. Louis Park, location.location.containedby, DeVry University-Minnesota', rating=2, reason='Contains location information but not directly related to the county of St. Louis Park')]


In [21]:
# execute_query("PREFIX ns: <http://rdf.freebase.com/ns/>\nSELECT DISTINCT ?x\nWHERE {\nFILTER (?x != ns:m.078w2)\nFILTER (!isLiteral(?x) OR lang(?x) = '' OR langMatches(lang(?x), 'en'))\nns:m.078w2 ns:influence.influence_node.influenced_by ?x .\n}\n")
entity = "m.01ps2h8"
relation = 'film.performance.actor'


for r in relation_w_score:
    entities = execute_query(query1)


['m.04601vx',
 'm.0ncf14h',
 'm.04hv0s5',
 'm.04tmj7k',
 'm.0csbx4z',
 'm.0gchffl',
 'm.0k2fhg',
 'm.0ncf4p_',
 'm.0jxpjg',
 'm.04p5_jq',
 'm.0gdjgzm',
 'm.0cgnnm_',
 'm.0cpkk3r',
 'm.0h10gwx',
 'm.02vcnpq',
 'm.0h4rhn3',
 'm.0crt0rh',
 'm.0jylf1',
 'm.0jw31h',
 'm.03lnwyf',
 'm.0cw0frt',
 'g.11b6ntbrrh',
 'm.0tlwfsx',
 'm.0k5s9k',
 'm.02vbcht',
 'm.04j3126',
 'm.0gg2g2_',
 'm.0k5scp',
 'm.010nvx_x',
 'm.03l_27l',
 'm.02tb39x',
 'm.02ws3bw',
 'm.02vcc2j',
 'g.11b6bnm19y',
 'm.02vb8dx',
 'm.045xysd',
 'm.04m58pp',
 'm.05t5kkc',
 'm.0cg2wlx',
 'm.0cg47_t',
 'm.0crsy7r',
 'm.0g5d2sy',
 'm.0gm1brq',
 'm.0gy9zn6',
 'm.0gy9zpf',
 'm.0j21lcf',
 'm.0j_w0g',
 'm.0jsry5',
 'm.0jsw4b',
 'm.0jt64l',
 'm.0jx0v6',
 'm.0jzd82',
 'm.0k5sfk',
 'm.0tlwgxy',
 'm.0v_0_0q']

In [3]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = "Given a question and the associated retrieved knowledge graph triplets (entity, relation, entity), you are asked to answer the question with these triplets and your knowledge."

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | model | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

The design of generative agents combines LLM with memory, planning, and reflection mechanisms to enable agents to behave conditioned on past experience and interact with other agents. Long-term memory provides the agent with the capability to retain and recall infinite information over extended periods. Short-term memory is utilized for in-context learning.


In [4]:
### Hallucination Grader


# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='yes')

In [5]:
### Answer Grader


# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='yes')

In [6]:
### Question Re-writer

# LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

"What is the role of memory in an agent's functioning?"

# Graph 

Capture the flow in as a graph.

## Graph state

In [7]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

In [8]:
### Nodes


def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

## Build Graph

The just follows the flow we outlined in the figure above.

In [10]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [11]:
from pprint import pprint

# Run
inputs = {"question": "Explain how the different types of agent memory work?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('Short-term memory is used for in-context learning in agents, allowing them '
 'to learn quickly. Long-term memory enables agents to retain and recall vast '
 'amounts of information over extended periods. Agents can also utilize '
 'external tools like APIs to access additional information beyond what is '
 'stored in their memory.')


In [12]:
inputs = {"question": "Explain how chain of thought prompting works?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
"Node 'generate':"
'\n---\n'
('Chain of thought prompting works by repeatedly prompting the model to ask '
 'follow-up questions to construct the thought process iteratively. This '
 'method can be combined with queries to search for relevant entities and '
 'content to add back into the context. It extends the thought process by '
 'exploring multiple reasoning possibilities at each step, creating a tree '
 'structure of thoughts.')


LangSmith Traces - 
 
* https://smith.langchain.com/public/55d6180f-aab8-42bc-8799-dadce6247d9b/r

* https://smith.langchain.com/public/1c6bf654-61b2-4fc5-9889-054b020c78aa/r

In [ ]:
python main_freebase.py \  # if you wanna use Wikidata as KG source, run main_wiki.py
--dataset cwq \ # dataset your wanna test, see ToG/data/README.md
--max_length 256 \ 
--temperature_exploration 0.4 \ # the temperature in exploration stage.
--temperature_exploration 0 \ # the temperature in reasoning stage.
--width 3 \ # choose the search width of ToG, 3 is the default setting.
--depth 3 \ # choose the search depth of ToG, 3 is the default setting.
--remove_unnecessary_rel True \ # whether removing unnecessary relations.
--LLM_type gpt-3.5-turbo \ # the LLM you choose
--opeani_api_keys sk-xxxx \ # your own api keys, if LLM_type == llama, this parameter would be rendered ineffective.
--num_retain_entity 5 \ # Number of entities retained during entities search.
--prune_tools llm \ # prune tools for ToG, can be llm (same as LLM_type), bm25 or sentencebert.

In [ ]:
outputs = []
count = 0
from tqdm.notebook import tqdm
with tqdm(total=len(dev_chain_data) , desc=f"Processing",ncols=1500) as pbar:
    while count < len(dev_chain_data):
        data = dev_chain_data[count]
        query = data['query']
        answer = ', '.join(data['answer'][:10])
        content = utility_chain.invoke({'query':query, 'output': answer})['text']
        try:
            content = eval(content)
            individual = content['individual_scores']
            overall = content['overall_scores']
            outputs.append({"qid": data['qid'], "query": query, "answer": data['answer'], "scores": [{"utility_score": overall, "individual_score": individual}],"score_type": 'utility'})
            count += 1
            pbar.update(1)
        except:
            print('error')
        if count % 10 == 0 and len(outputs):
            print(f'Saving {count}')
            save_to_json(outputs, './output/generate_data/generate_cwq_utility_0201.json')
            outputs = []
    


In [ ]:
# 新的reflection token1
few_shot_is_useful = FewShotPromptTemplate(
        examples=[{
            "query": "What is the name of Justin Bieber's brother?",
            "output": "Jaxon Bieber",
            "rating": """{{"individual_scores": {{"Jaxon Bieber": "[Fully Useful]"}}, "overall_scores": "[Utility:5]", "explanation": "The answer Jaxon Bieber provides useful information for the query, so its individual score should be [Fully Useful]. Generally, the query is fully answered, so the overall score should be [Utility:5]."}}"""},
                  {
            "query": "In which year did Donald Trump win the US presidential election?",
            "output": "2024",
            "rating": """{{"individual_scores": {{"2024": "[Fully Useful]"}}, "overall_scores": "[Utility:4]", "explanation": "2024 is a correct answer to the query, so its individual score should be [Fully Useful]. Generally, the query is answered correctly, but there should be two correct answers: 2016 and 2024, while the output only provides one. So the overall score should be [Utility:4]."}}"""},
                  {
            "query": "I want to visit Renmin University of China, but I don't know where it is. Tell me its address.",
            "output": "Beijing, Haidian District",
            "rating":  """{{"individual_scores": {{"Beijing": "[Partially Useful]","Haidian District": "[Partially Useful]"}}, "overall_scores": "[Utility:3]", "explanation": "Both Beijing and Haidian District are correct but too vague to help locate the university campus. So the individual scores should be [Partially Useful], and the overall score should be [Utility:3]."}}"""},
                  {
            "query": "Who was the Governor of Michigan in July 2017?",
            "output": "Gretchen Whitmer",
            "rating": """{{"individual_scores": {{"Gretchen Whitmer": "[Not Useful]"}}, "overall_scores": "[Utility:2]", "explanation": "The output is still talking about the governor of Michigan and providing some correct information. But it does not answer who the governor was in July 2017 (Gretchen Whitmer has been serving as the governor from 2019), not providing useful information for the query. So the individual score of Gretchen Whitmer should be [Not Useful], and the overall score should be [Utility:2]."}}"""},
                  {
            "query": "What are the advantages of Python in data analysis?",
            "output": "Guido van Rossum, Portugal",
            "rating": """{{"individual_scores": {{"Guido van Rossum": "[Not Useful]", "Portugal": "[Not Useful]"}}, "overall_scores": "[Utility:1]", "explanation": "The output is completely irrelevant to the query. So the individual scores should be [Not Useful], and the overall score should be [Utility:1]."}}"""}],
        example_prompt=PromptTemplate.from_template("""
Query: {query}
Answer: {output}
Rating: {rating}
"""),
        prefix="""You will be given a query and the answers, where the answers may consist of one or more individual answers, separated by commas(,). 
Your task is to generate a **rating** to evaluate whether the answer is a useful response to the query. The rating provide an individual score for each individual answer, and then give an overall score for the entire output.
Use the following entailment scale to give individual score(s):
[Fully Useful]: The individual answer provides correct and useful information for the query.
[Partially Useful]: The individual answer may provide some correct information that is helpful in addressing the query, but it has some flaws, such as lacking specificity.
[Not Useful]: The individual answer provides incorrect information or is irrelevant to the query.
Use the following entailment scale to give an overall score:
[Utility:5]: Generally, the output provides a complete, highly detailed, and informative response to the query, fully satisfying the information needs.
[Utility:4]: Generally, the output mostly fulfills the need in the query and provides helpful answers, while there can be some minor improvements, such as discussing more detailed information or providing additional correct answers beyond the current output.
[Utility:3]: Generally, the output is correct and acceptable, but there are obvious problems, such as being too vague or not specific enough, limiting its helpfulness in addressing the query. 
[Utility:2]: Generally, the output still discusses the topic of the query, but it is incorrect or does not actually meet the requirements of the query.
[Utility:1]: Generally, the output is completely irrelevant to the query or does not give an answer in the end.
""",
        suffix=
"""
Query: {query}
Answers: {output}
Rating:
 """,
        input_variables=["query", "output"],
)


utility_chain = LLMChain(llm=model, prompt=few_shot_is_useful, verbose=False)